In [42]:
import modin
NUM_STOCKS=10
import yahoo_fin as yf
from yahoo_fin.stock_info import get_data
import time
import datetime
import pandas as pd
from yahoo_fin import stock_info as si

import tensorflow as tf 
import numpy as np
from tensorflow.keras import layers

import concurrent.futures

# getting data

things we did to clean data:

1. shamelessly copy:
https://levelup.gitconnected.com/how-to-get-all-stock-symbols-a73925c16a1b

2. we removed the tickers with $

3. we added exception handeling that tries similer strings to the input (note that this is not implemented on the set itself but when fetching, leading to multiple requsts and potentialy to errors)
4.  we just drop exceptions that are found when getting the data from online  (these are diffrent every day)

# how are we keeping consistency? 
since the avilble stocks may change depending on the source of the data some extra steps need to be done in order to
alow saves to work seemlesly 


basic plan:

1. clean the data as much as possible 

2. save the embedings weights with their ticker string

3. backup everything on git to alow us to fix it



In [2]:
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )
df4 = pd.DataFrame( si.tickers_other() )

sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
sym4 = set( symbol for symbol in df4[0].values.tolist() )

symbols = set.union( sym1, sym2, sym3, sym4 )
del(sym1, sym2, sym3, sym4,df1, df2, df3, df4)

In [3]:
my_list = ['W', 'R', 'P', 'Q']
del_set = set()
sav_set = set()

for symbol in symbols:
    if (len( symbol ) > 4 and symbol[-1] in my_list or "$" in symbol):
        del_set.add( symbol )
    else:
        sav_set.add( symbol )
sav_set.discard("") 

ticker_list=list(sav_set)

print( f'Removed {len( del_set )} unqualified stock symbols...' )
print( f'There are {len( sav_set )} qualified stock symbols...' )

Removed 1582 unqualified stock symbols...
There are 10623 qualified stock symbols...


In [4]:
start='01/01/2020' 
end='09/19/2022'

In [5]:
import concurrent.futures 

def get_seiries(ticker,start,end,entry="adjclose"):
    try:
        raw = get_data(ticker.split(".")[0], start_date=start, end_date=end)[entry]
    except: 
        try:
            raw = get_data(ticker.replace(".","-"), start_date=start, end_date=end)[entry]
        except:
            raw = get_data(ticker[0:4], start_date=start, end_date=end)[entry]
    return raw/raw[0]

def get_df(l,start,end,entry="adjclose"):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        x=[executor.submit(lambda x: get_seiries(x,start,end,entry=entry),name)for name in l]
        x=[a.result() for a in x]
    for i,c in enumerate(x):
        c.name=l[i]
    return pd.DataFrame(x).transpose()

def tester_func(name,start=start,end=end,entry="adjclose"):
    try:
        c= get_seiries(name,start,end,entry)
        c.name=name
        return (True,c)
    except:
        return (False,name)

In [6]:
mini=ticker_list
errors=[]
data=[]
c=0
x=list(range(len(mini)))
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in range(len(mini)):
        x[i]=executor.submit(tester_func,mini[i])
    
    print("thread setup complete")
    
    for i in range(len(mini)):
        if(i%100==0):
            print(i)
        r=x[i].result()
        if (r[0]): 
            data.append(r[1])
        else:
            errors.append(r[1])
            c+=1
            print(f"error number {c}: name={errors[-1]}")

print("done")

thread setup complete
0
100
200
300
error number 1: name=EMP
400
error number 2: name=XTWY
500
600
error number 3: name=ZCZZT
error number 4: name=KDIV
700
800
error number 5: name=RCA
900
1000
error number 6: name=ZXYZ.A
1100
error number 7: name=ATEST.C
1200
error number 8: name=ZXZZT
1300
1400
1500
1600
1700
error number 9: name=DHCNL
1800
1900
error number 10: name=ZBZX
2000
2100
2200
2300
error number 11: name=ZJZZT
2400
2500
error number 12: name=CBO
2600
2700
error number 13: name=SFB
error number 14: name=CTEST.S
2800
error number 15: name=ZXIET
2900
3000
3100
error number 16: name=ZVV
3200
error number 17: name=ATEST.A
3300
error number 18: name=ATEST
3400
3500
3600
3700
error number 19: name=CTEST.O
3800
3900
error number 20: name=GRP.U
4000
error number 21: name=CTEST.G
4100
4200
4300
error number 22: name=IGZ
4400
error number 23: name=IBO
4500
error number 24: name=ZBZZT
4600
4700
4800
4900
error number 25: name=XTWO
5000
5100
5200
error number 26: name=ZVZZT
5300
5400
550

In [43]:
data_df=pd.DataFrame(data).transpose()
data_df

,UTAAU,FVAL,RYU,PKBK,PLM,PNF,FNCB,QGRO,UXIN,ONEV,...,CLVT,FDP,MYNZ,BXMX,NQP,REMG,MDJH,FSBW,OAIE,SCOB
2020-01-02,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,NaN,1.000000,1.000000,NaN,1.000000,1.000000,NaN,NaN
2020-01-03,NaN,0.993187,1.001432,0.988746,1.000000,0.984150,0.986998,0.992088,0.986900,0.994473,...,1.004086,1.004358,NaN,0.990525,1.001412,NaN,0.897288,0.993946,NaN,NaN
2020-01-06,NaN,0.993868,1.001623,0.991157,1.115385,0.987752,0.986998,0.994725,1.082969,0.993410,...,1.004670,0.995352,NaN,0.990525,1.004237,NaN,0.983051,1.001434,NaN,NaN
2020-01-07,NaN,0.992400,1.000286,0.987138,1.115385,1.005764,0.960993,0.996923,1.048035,0.991907,...,1.002335,0.985764,NaN,0.985423,1.007768,NaN,1.003390,0.981361,NaN,NaN
2020-01-08,NaN,0.996253,1.000286,0.997990,1.269231,1.003602,0.964539,1.005275,1.052402,0.993988,...,0.996497,0.994771,NaN,0.992711,1.008474,NaN,0.958983,0.981520,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,1.00499,1.239907,1.263158,1.073983,1.292308,0.770535,0.982090,1.390525,0.313100,1.253491,...,0.711617,0.758444,0.338624,1.155949,0.925426,0.802848,0.654237,0.958590,1.045259,1.00000
2022-09-13,1.00499,1.182837,1.229249,1.038421,1.253846,0.742903,0.965929,1.329366,0.314410,1.207910,...,0.687099,0.759042,0.339758,1.122516,0.919188,0.782947,0.661017,0.958590,1.020960,1.00202
2022-09-14,1.00499,1.186875,1.240453,1.039369,1.276923,0.732640,0.933607,1.337315,0.301310,1.199983,...,0.676007,0.741710,0.331444,1.123351,0.911392,0.786589,0.633898,0.951027,1.014845,1.00202
2022-09-15,1.00499,1.175569,1.212295,1.043162,1.269231,0.718429,1.029329,1.318327,0.299127,1.193022,...,0.636894,0.734537,0.329592,1.108307,0.900477,0.778146,0.650847,0.950040,0.997707,1.00303


In [44]:
train_df=data_df.iloc[:-69]
test_df=data_df.iloc[-69:]

test_df

,UTAAU,FVAL,RYU,PKBK,PLM,PNF,FNCB,QGRO,UXIN,ONEV,...,CLVT,FDP,MYNZ,BXMX,NQP,REMG,MDJH,FSBW,OAIE,SCOB
2022-06-09,1.039920,1.250674,1.170057,1.116184,1.130769,0.756324,0.927391,1.331574,0.193450,1.228102,...,0.843549,0.736928,0.414588,1.143411,0.965187,0.843179,0.542373,0.987200,NaN,0.990909
2022-06-10,1.039920,1.214064,1.160241,1.086311,1.111539,0.750798,0.944795,1.286754,0.181659,1.205335,...,0.815528,0.726768,0.414210,1.123351,0.951153,0.834437,0.633898,0.970757,NaN,0.992929
2022-06-13,1.008982,1.162110,1.103925,1.027041,1.019231,0.740535,0.943552,1.222063,0.174672,1.164331,...,0.800934,0.707642,0.351474,1.078217,0.931663,0.806755,0.633898,0.962865,NaN,0.991919
2022-06-14,1.007984,1.160764,1.072990,1.021351,1.023077,0.728692,0.934850,1.225154,0.177729,1.156702,...,0.771162,0.711228,0.336735,1.043948,0.905935,0.817715,0.603390,0.956288,NaN,0.990909
2022-06-15,1.016966,1.177454,1.078642,1.065922,1.065385,0.739745,1.024357,1.247233,0.187336,1.165642,...,0.798599,0.708539,0.388889,1.058993,0.903596,0.831225,0.545763,0.954973,NaN,0.990404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,1.004990,1.239907,1.263158,1.073983,1.292308,0.770535,0.982090,1.390525,0.313100,1.253491,...,0.711617,0.758444,0.338624,1.155949,0.925426,0.802848,0.654237,0.958590,1.045259,1.000000
2022-09-13,1.004990,1.182837,1.229249,1.038421,1.253846,0.742903,0.965929,1.329366,0.314410,1.207910,...,0.687099,0.759042,0.339758,1.122516,0.919188,0.782947,0.661017,0.958590,1.020960,1.002020
2022-09-14,1.004990,1.186875,1.240453,1.039369,1.276923,0.732640,0.933607,1.337315,0.301310,1.199983,...,0.676007,0.741710,0.331444,1.123351,0.911392,0.786589,0.633898,0.951027,1.014845,1.002020
2022-09-15,1.004990,1.175569,1.212295,1.043162,1.269231,0.718429,1.029329,1.318327,0.299127,1.193022,...,0.636894,0.734537,0.329592,1.108307,0.900477,0.778146,0.650847,0.950040,0.997707,1.003030


In [45]:
ticker_list=data_df.columns
print(f"{len(symbols)-len(ticker_list)} stocks were droped")

1628 stocks were droped


# preprocessing 


1. ticker: this is also used to select which stocks come to the model

2. price: we take the ratio: (importantly we did drop some potentialy important indicators)

3. date: the day and month are floating point numbers. while year and day of the week are embedings.



In [46]:
look=layers.StringLookup(vocabulary=ticker_list,num_oov_indices=0)
inv_look=layers.StringLookup(vocabulary=ticker_list,invert=True,num_oov_indices=0)
tick_embed=layers.Embedding(len(ticker_list)+1,512)#,embeddings_constraint= tf.keras.constraints.unit_norm)

In [47]:
inputs=layers.Input([],dtype=tf.string)
x=look(inputs)
x=tick_embed(x)
x=layers.UnitNormalization()(x)
ticker_model=tf.keras.Model(inputs,x)
ticker_model.compile()

In [48]:
dataset=tf.data.Dataset.from_tensor_slices(ticker_list)
exmple=dataset.batch(17).take(1).get_single_element()
exmple

<tf.Tensor: shape=(17,), dtype=string, numpy=
array([b'UTAAU', b'FVAL', b'RYU', b'PKBK', b'PLM', b'PNF', b'FNCB',
       b'QGRO', b'UXIN', b'ONEV', b'NORW', b'RE', b'DUOT', b'RIBT',
       b'IAG', b'HIPO', b'UI'], dtype=object)>

In [49]:
#data_df[exmple.numpy().astype('U13')]

In [50]:
def thread_distribute(l,f,n=1):
    #print(type(l))
    if(n==0):
        return f(l)
    
    with concurrent.futures.ThreadPoolExecutor() as executor: 
        l=[executor.submit(lambda x: thread_distribute(x,f,n-1),x) for x in l]
        l= [x.result() for x in l]
    return l

thread_distribute([range(5) for i in range(2)],lambda x: range(x),2)

[[range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)],
 [range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)]]

In [51]:
a[0][0].index

NameError: name 'a' is not defined

In [52]:
def time_encode(a):
    #a=a.index
    #print(type(a))
    return (a.day_of_week,a.day/30,a.month/12,(a.year-2020))
    
time_encode(data_df.index[0])

(3, 0.06666666666666667, 0.08333333333333333, 0)

In [53]:
def choose_stocks_df(x,data_df=train_df):
    a=ticker_model(ticker_list)
    b=ticker_model(x)

    b=b@tf.transpose(a)
    b=tf.nn.top_k(b,NUM_STOCKS)


    tickers=inv_look([list(x) for x in b[1].numpy()]).numpy().astype('U13')
    
    
    return [pd.DataFrame([data_df[x] for x in sub]).transpose().dropna() for sub in tickers]

v=choose_stocks_df(exmple)
v[0]

,UTAAU,VTEB,AB,PFMT,NYMTL,CWS,VEEE,OXLCM,MCR,MPX
2021-12-02,1.000000,1.054594,2.001236,2.366072,0.975455,1.349973,0.413885,1.089973,1.164135,0.903909
2021-12-03,0.999002,1.055746,2.000843,2.294643,0.977435,1.331377,0.387183,1.092565,1.158578,0.891272
2021-12-06,0.999002,1.054594,2.065843,2.348214,0.979810,1.348339,0.440587,1.090405,1.180805,0.926209
2021-12-07,1.002994,1.054210,2.083177,2.553571,0.978306,1.362138,0.457944,1.089973,1.187751,0.961146
2021-12-08,1.001996,1.054017,2.039843,2.625000,0.979612,1.369239,0.507343,1.094725,1.183584,0.956686
...,...,...,...,...,...,...,...,...,...,...
2022-06-02,1.007485,0.976248,1.756433,2.366072,0.903405,1.202706,0.419092,1.094077,0.955936,0.828445
2022-06-03,1.010978,0.976440,1.730097,2.303571,0.906968,1.194882,0.420561,1.097317,0.953099,0.825443
2022-06-06,1.006387,0.974520,1.757243,2.357143,0.898654,1.200616,0.417891,1.096367,0.946008,0.799179
2022-06-07,1.009980,0.972407,1.792899,2.285714,0.903009,1.210450,0.413885,1.094293,0.947426,0.821691


# time handeling 

problems with the data in curent time format:

1. there are null entries especialy from before a company was made 
2. the time diffrence between entries isnt clean

In [54]:
mask=np.logical_not(data_df.isna().to_numpy())
mask

array([[False,  True,  True, ...,  True, False, False],
       [False,  True,  True, ...,  True, False, False],
       [False,  True,  True, ...,  True, False, False],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [55]:
# need to add the tickers strings/ints
def get_xs(v):
    t=[x.index for x in v]
    #for x in t:
     #   print(type(x))
    t=thread_distribute(t,time_encode,2)
    #t=[[x[i] for x in t] for i in range(4)]
    #print("time encoded")
    t=[thread_distribute(t,lambda x: x[i],2) for i in range(4)]
    #thread_distribute(t,len,2)
    t=[tf.ragged.constant(x) for x in t]
    
    x=thread_distribute(v,lambda x: x.to_numpy(),1)
    vals=tf.concat(x,axis=0)
    lengths=[len(s) for s in x]
    x=tf.RaggedTensor.from_row_lengths(vals,lengths)
    #[x.shape for x in x]
    
    "padding"
    x=(x+1).to_tensor()-1
    t=[(x+1).to_tensor()-1 for x in t] 
    
    x=tf.transpose(x,[2,0,1])
    t.extend(x) 
    
    tickers=tf.constant([x.transpose().index for x in v])
    t.append(tickers)
    return t

xs=get_xs(v)
[(x.shape,x.dtype) for x in xs]

[(TensorShape([17, 375]), tf.int32),
 (TensorShape([17, 375]), tf.float32),
 (TensorShape([17, 375]), tf.float32),
 (TensorShape([17, 375]), tf.int32),
 (TensorShape([17, 375]), tf.float64),
 (TensorShape([17, 375]), tf.float64),
 (TensorShape([17, 375]), tf.float64),
 (TensorShape([17, 375]), tf.float64),
 (TensorShape([17, 375]), tf.float64),
 (TensorShape([17, 375]), tf.float64),
 (TensorShape([17, 375]), tf.float64),
 (TensorShape([17, 375]), tf.float64),
 (TensorShape([17, 375]), tf.float64),
 (TensorShape([17, 375]), tf.float64),
 (TensorShape([17, 10]), tf.string)]

In [56]:
#[len(x) for x in v]

In [57]:
def sample_seq(x,LEN=100):
    l=len(x)
    if(l<LEN):
        return x 
    
    start=np.random.randint(l-LEN)
    return x.iloc[start:start+LEN]
sample_seq(v[-1]) 
sample_seq(v[0])

,UTAAU,VTEB,AB,PFMT,NYMTL,CWS,VEEE,OXLCM,MCR,MPX
2021-12-28,0.998004,1.055554,1.931508,2.071429,0.982581,1.418542,0.555407,1.092997,1.201320,0.923236
2021-12-29,1.000000,1.054978,1.920872,2.044643,0.981789,1.412781,0.579439,1.090837,1.211109,0.934386
2021-12-30,0.999002,1.054786,1.930720,2.169643,0.969121,1.423848,0.562083,1.091701,1.209711,0.934386
2021-12-31,1.006487,1.054402,1.924023,2.151786,0.971892,1.429555,0.535381,1.091701,1.191530,0.929182
2022-01-03,0.997804,1.053825,1.919296,2.232143,0.972447,1.403939,0.514019,1.093429,1.192928,0.935873
...,...,...,...,...,...,...,...,...,...,...
2022-05-13,1.021058,0.946292,1.547768,2.098214,0.883215,1.172294,0.420561,1.089109,0.916224,0.855459
2022-05-16,1.028443,0.947060,1.533182,2.008929,0.863816,1.172669,0.445928,1.090837,0.921897,0.855459
2022-05-17,1.028443,0.944372,1.602467,1.964286,0.855107,1.188424,0.453939,1.090102,0.917642,0.903485
2022-05-18,1.028443,0.943412,1.580587,1.812500,0.853127,1.146865,0.447263,1.093429,0.920478,0.855459


In [58]:
base=[sample_seq(x) for x in v]
y=[s.iloc[1:,0] for s in base]
x=[s[:-1] for s in base]
x=get_xs(x)
x

[<tf.Tensor: shape=(17, 99), dtype=int32, numpy=
 array([[ 2,  3,  4, ...,  1,  2,  3],
        [ 1,  2,  3, ...,  0,  1,  2],
        [ 0,  1,  2, ..., -1, -1, -1],
        ...,
        [ 4,  0,  1, ...,  4,  0,  1],
        [ 1,  2,  3, ...,  0,  1,  2],
        [ 3,  4,  1, ...,  3,  4,  0]])>,
 <tf.Tensor: shape=(17, 99), dtype=float32, numpy=
 array([[ 0.16666663,  0.20000005,  0.23333335, ...,  0.79999995,
          0.83333325,  0.8666667 ],
        [ 0.33333337,  0.36666667,  0.39999998, ...,  0.9       ,
          0.9333334 ,  0.9666667 ],
        [ 0.13333333,  0.16666663,  0.20000005, ..., -1.        ,
         -1.        , -1.        ],
        ...,
        [ 0.33333337,  0.4333334 ,  0.4666667 , ...,  0.9666667 ,
          0.06666672,  0.10000002],
        [ 0.9333334 ,  0.9666667 ,  1.        , ...,  0.5333333 ,
          0.5666666 ,  0.6       ],
        [ 0.4333334 ,  0.4666667 ,  0.6       , ...,  0.06666672,
          0.10000002,  0.20000005]], dtype=float32)>,
 <tf.Te

In [59]:
def get_numbers(tickers):
    v=choose_stocks_df(tickers)
    base=[sample_seq(x) for x in v]
    
    y=[s.iloc[1:,0].to_numpy() for s in base]
    y=tf.ragged.constant(y)
    y=(y+1).to_tensor()-1
    
    x=[s[:-1] for s in base]
    x=get_xs(x)
    #x.append(tickers)
    x.append(y)
    return x
t=time.time()
n=get_numbers(exmple)
print(time.time()-t)

0.9928314685821533


In [60]:
specs=[tf.TensorSpec.from_tensor(x) for x in n]
s=[list(x.shape) for x in specs]
for x in s:
    x[0]=None
specs=[tf.TensorSpec(s[i],specs[i].dtype) for i in range(len(specs))]
specs

[TensorSpec(shape=(None, 99), dtype=tf.int32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.int32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 10), dtype=tf.string, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None)]

In [61]:
def gather_x(*args):
    return args[:-1]

def gather_y(*args):
    return args[-1]

gather_x(1,2,3,4),gather_y(1,2,3,4)

((1, 2, 3), 4)

# train dataset 
because of the need to use the dataset as a refrence type in the pandas df we are forcing tensorflow to run the whole thing eagerly. 

it shouldnt be a major issue since most the heavy lifting is done in pandas which is optimized c code 

In [62]:
#need to do training and test set (test set should look diffrent)
train_dataset=dataset.shuffle(1000).batch(32).map(lambda y: tf.py_function(get_numbers,inp=[y],Tout=specs))

dataset_x=train_dataset.map(gather_x)
dataset_y=train_dataset.map(gather_y)
train_dataset=tf.data.Dataset.zip((dataset_x,dataset_y))

x=train_dataset.take(1).get_single_element()
x

((<tf.Tensor: shape=(32, 99), dtype=int32, numpy=
  array([[-1, -1, -1, ..., -1, -1, -1],
         [ 2,  3,  4, ...,  2,  3,  4],
         [-1, -1, -1, ..., -1, -1, -1],
         ...,
         [ 0,  1,  2, ...,  4,  0,  1],
         [-1, -1, -1, ..., -1, -1, -1],
         [-1, -1, -1, ..., -1, -1, -1]])>,
  <tf.Tensor: shape=(32, 99), dtype=float32, numpy=
  array([[-1.        , -1.        , -1.        , ..., -1.        ,
          -1.        , -1.        ],
         [ 0.39999998,  0.4333334 ,  0.4666667 , ...,  0.0333333 ,
           0.06666672,  0.10000002],
         [-1.        , -1.        , -1.        , ..., -1.        ,
          -1.        , -1.        ],
         ...,
         [ 0.66666675,  0.70000005,  0.73333335, ...,  0.13333333,
           0.23333335,  0.26666665],
         [-1.        , -1.        , -1.        , ..., -1.        ,
          -1.        , -1.        ],
         [-1.        , -1.        , -1.        , ..., -1.        ,
          -1.        , -1.        ]], dt

In [63]:
[tf.TensorSpec.from_tensor(x) for x in x[0]]

[TensorSpec(shape=(32, 99), dtype=tf.int32, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.int32, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 10), dtype=tf.string, name=None)]

In [64]:
def cut23(x):
    return {"time":x[:4],"prices":x[4:NUM_STOCKS],"tickers":x[NUM_STOCKS]}

cut23(x[0])

{'time': (<tf.Tensor: shape=(32, 99), dtype=int32, numpy=
  array([[-1, -1, -1, ..., -1, -1, -1],
         [ 2,  3,  4, ...,  2,  3,  4],
         [-1, -1, -1, ..., -1, -1, -1],
         ...,
         [ 0,  1,  2, ...,  4,  0,  1],
         [-1, -1, -1, ..., -1, -1, -1],
         [-1, -1, -1, ..., -1, -1, -1]])>,
  <tf.Tensor: shape=(32, 99), dtype=float32, numpy=
  array([[-1.        , -1.        , -1.        , ..., -1.        ,
          -1.        , -1.        ],
         [ 0.39999998,  0.4333334 ,  0.4666667 , ...,  0.0333333 ,
           0.06666672,  0.10000002],
         [-1.        , -1.        , -1.        , ..., -1.        ,
          -1.        , -1.        ],
         ...,
         [ 0.66666675,  0.70000005,  0.73333335, ...,  0.13333333,
           0.23333335,  0.26666665],
         [-1.        , -1.        , -1.        , ..., -1.        ,
          -1.        , -1.        ],
         [-1.        , -1.        , -1.        , ..., -1.        ,
          -1.        , -1.      

# validation dataset 
on infrance the best results would probably be achived by runing on every time entry with at least some memory of the past

that means there would need to be some precomputation which we can make diffrent for timesetep. 

this is rediclously slow and combersom to write so our validation is just gona look mostly like the training. 
after choosing the stocks we would want to work with we are going to think of test behivior 

we may want to just train a new model from scratch only on that subset.

In [65]:
def get_val_numbers(tickers):
    base=choose_stocks_df(tickers,test_df)
    
    y=[s.iloc[1:,0].to_numpy() for s in base]
    y=tf.ragged.constant(y)
    y=(y+1).to_tensor()-1
    
    x=[s[:-1] for s in base]
    x=get_xs(x)
    #x.append(tickers)
    x.append(y)
    return x
t=time.time()
n=get_val_numbers(exmple)
print(time.time()-t)

0.7330348491668701


In [66]:
#need to do training and test set (test set should look diffrent)
val_dataset=dataset.shuffle(1000).batch(32).map(lambda y: tf.py_function(get_val_numbers,inp=[y],Tout=specs))

dataset_x=val_dataset.map(gather_x)
dataset_y=val_dataset.map(gather_y)
val_dataset=tf.data.Dataset.zip((dataset_x,dataset_y))

x=val_dataset.take(1).get_single_element()

In [67]:
[tf.TensorSpec.from_tensor(x) for x in x[0]]

[TensorSpec(shape=(32, 68), dtype=tf.int32, name=None),
 TensorSpec(shape=(32, 68), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 68), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 68), dtype=tf.int32, name=None),
 TensorSpec(shape=(32, 68), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 68), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 68), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 68), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 68), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 68), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 68), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 68), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 68), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 68), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 10), dtype=tf.string, name=None)]

# the model

**the dataset is dependent on the model!!!** this means we have some considerations on how it should look like:

* there has to be an attention mechanisem on embeding space. (using only the sampling embedings dot product) 
* embedings convergnce and stabilety greatly effects the models training. (we may want to consider reverse regularization)
*  at least some part of the model should be seprable by stock, in order to reduce noise from data sampeling
* it would defintly help if we add residual conections to the embedings to alow gradients to flow to the embedings

*side note: the padding used is -1, seems like it should propgate to the loss*


In [68]:
#setting up the inputs
inputs=[layers.Input([None,],dtype=x.dtype) for x in specs[:-2]]
inputs.append(layers.Input(specs[-2].shape[1:],dtype=specs[-2].dtype,name="tickers"))

#for testing
#inputs=numbers[:-1]

#time encoding
time_depndent=[tf.expand_dims(x,-1) for x in inputs[:-1]] 
time_depndent=[layers.Masking(-1)(x) for x in time_depndent] 


day_embeding=layers.Embedding(7,7)(time_depndent[0])
year_embeding=layers.Embedding(3,5)(time_depndent[3]) 

day_embeding=layers.TimeDistributed(layers.Reshape([7]))(day_embeding)
year_embeding=layers.TimeDistributed(layers.Reshape([5]))(year_embeding)

t=layers.Concatenate(-1)(time_depndent[1:3])
t=layers.Dense(20)(t) 

time_encoding=layers.Concatenate(-1)([day_embeding,t,year_embeding])
time_encoding

<KerasTensor: shape=(None, None, 32) dtype=float32 (created by layer 'concatenate_153')>

In [69]:
#tickers=look(inputs[-1])
#tickers=tick_embed(tickers)
tickers=layers.Reshape([NUM_STOCKS,1])(inputs[-1])
tickers=layers.TimeDistributed(ticker_model)(tickers)
tickers=layers.Reshape([NUM_STOCKS,512])(tickers)
origin=tf.expand_dims(tickers[:,0],-2)
origin=layers.Permute([2,1])(origin)
attention=tickers@origin
attention=layers.Permute([2,1])(attention)
attention=tf.nn.softmax(attention)
attention

<KerasTensor: shape=(None, 1, 10) dtype=float32 (created by layer 'tf.nn.softmax_1')>

In [70]:
comp=layers.Dense(33)(tickers)
comp

<KerasTensor: shape=(None, 10, 33) dtype=float32 (created by layer 'dense_3')>

In [71]:
"need to add stock conditioning"
prices=time_depndent[4:4+NUM_STOCKS]
s=[layers.Concatenate(-1)([time_encoding,x]) for x in prices]

#adding ticker information (while not alowing it to effect the mask)
#masks=[x._keras_mask for x in s] 
s=[layers.Add()([s[i],comp[:,i]]) for i in range(len(s))]

stock_layer=layers.LSTM(16,return_sequences=True)
s=[stock_layer(s[i]) for i in range(len(s))]#,mask=masks[i])
stock_layer=layers.GRU(16,return_sequences=True)
s=[stock_layer(x) for x in s]

s=[tf.expand_dims(x,-2) for x in s]
s=layers.Concatenate(-2)(s)
s

<KerasTensor: shape=(None, None, 10, 16) dtype=float32 (created by layer 'concatenate_164')>

In [72]:
sumorized=attention@tf.transpose(s,[1,0,2,3])
sumorized=tf.transpose(sumorized,[1,0,2,3])
sumorized=layers.TimeDistributed(layers.Reshape([16]))(sumorized)
sumorized

<KerasTensor: shape=(None, None, 16) dtype=float32 (created by layer 'time_distributed_6')>

In [73]:
x=layers.LSTM(8,return_sequences=True)(sumorized)
x=layers.LayerNormalization()(x)
x=layers.Permute([2,1])(x)
x=layers.GlobalMaxPool1D()(x)
x

<KerasTensor: shape=(None, None) dtype=float32 (created by layer 'global_max_pooling1d')>

In [75]:
model=tf.keras.Model(inputs,x)
#model(numbers[:-1])
model.compile(optimizer="adam",loss="mae")
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_57 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_58 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_59 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_60 (InputLayer)          [(None, None)]       0           []                               
                                                                                            

 tf.expand_dims_65 (TFOpLambda)  (None, None, 1)     0           ['input_67[0][0]']               
                                                                                                  
 tf.expand_dims_66 (TFOpLambda)  (None, None, 1)     0           ['input_68[0][0]']               
                                                                                                  
 tf.expand_dims_67 (TFOpLambda)  (None, None, 1)     0           ['input_69[0][0]']               
                                                                                                  
 tf.expand_dims_68 (TFOpLambda)  (None, None, 1)     0           ['input_70[0][0]']               
                                                                                                  
 concatenate_153 (Concatenate)  (None, None, 32)     0           ['time_distributed_3[0][0]',     
                                                                  'dense_2[0][0]',                
          

                                                                  'masking_66[0][0]']             
                                                                                                  
 tf.__operators__.getitem_160 (  (None, 33)          0           ['dense_3[0][0]']                
 SlicingOpLambda)                                                                                 
                                                                                                  
 concatenate_163 (Concatenate)  (None, None, 33)     0           ['concatenate_153[0][0]',        
                                                                  'masking_67[0][0]']             
                                                                                                  
 tf.__operators__.getitem_161 (  (None, 33)          0           ['dense_3[0][0]']                
 SlicingOpLambda)                                                                                 
          

                                                                                                  
 tf.expand_dims_70 (TFOpLambda)  (None, None, 1, 16)  0          ['gru_3[0][0]']                  
                                                                                                  
 tf.expand_dims_71 (TFOpLambda)  (None, None, 1, 16)  0          ['gru_3[1][0]']                  
                                                                                                  
 tf.expand_dims_72 (TFOpLambda)  (None, None, 1, 16)  0          ['gru_3[2][0]']                  
                                                                                                  
 tf.expand_dims_73 (TFOpLambda)  (None, None, 1, 16)  0          ['gru_3[3][0]']                  
                                                                                                  
 tf.expand_dims_74 (TFOpLambda)  (None, None, 1, 16)  0          ['gru_3[4][0]']                  
          

In [76]:
numbers=get_numbers(exmple.numpy())
x=numbers[:-1]
y=numbers[-1]

In [77]:
model.fit(train_dataset,epochs=5)

Epoch 1/5
  5/331 [..............................] - ETA: 14:51 - loss: 0.9637

InvalidArgumentError: Graph execution error:

ValueError: high <= 0
Traceback (most recent call last):

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 268, in __call__
    return func(device, token, args)

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 146, in __call__
    outputs = self._call(device, args)

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 153, in _call
    ret = self._func(*args)

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 812, in wrapped_func
    out = func(*structured_inp)

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\Owner\AppData\Local\Temp/ipykernel_1408/3836857378.py", line 3, in get_numbers
    base=[sample_seq(x) for x in v]

  File "C:\Users\Owner\AppData

In [78]:
d=train_dataset.as_numpy_iterator()

In [83]:
for i in range(len(train_dataset)):
    print(i)
    error_maker=d.next()
    model.train_on_batch(error_maker)

0


ValueError: in user code:

    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\input_spec.py", line 200, in assert_input_compatibility
        raise ValueError(f'Layer "{layer_name}" expe

In [88]:
len(error_maker[1])
error_maker[1]

array([[ 0.01655307,  0.01655307,  0.01752678, ..., -1.        ,
        -1.        , -1.        ],
       [ 0.90534258,  0.91476835,  0.8695248 , ...,  1.08437055,
         1.12746121,  1.13124944],
       [ 1.0442689 ,  1.04242881,  1.05520076, ..., -1.        ,
        -1.        , -1.        ],
       ...,
       [ 1.17322823,  1.16866311,  1.17532058, ...,  1.0404602 ,
         1.02448238,  1.06423671],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ]])